In [2]:
import geopandas as gpd
import pandas as pd
import datetime
from cityiq import Config, CityIq
from cityiq.api import Asset, Location
from cityiq.scrape import EventScraper

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt


%load_ext autoreload
%autoreload 2

tz = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
f = '/Users/eric/proj/virt-proj/data-project/sdrdl-data-projects/sandiego.gov/predix.io/prod-credentials.yaml'
config = Config(f)
ciq = CityIq(config, start_time='20190101')

In [3]:
loc_rec = list(ciq.locations)

In [4]:
locations = gpd.GeoDataFrame(ciq.locations_dataframe, geometry='geometry')
assets = gpd.GeoDataFrame(ciq.asset_dataframe, geometry='geometry')

locations.head()

locationUid  locationType parentLocationUid  \
0    004361eb       WALKWAY          004361eb   
1    00456472  TRAFFIC_LANE          00456472   
2    0051796c       WALKWAY          0051796c   
3    00537bf3  TRAFFIC_LANE          00537bf3   
4    005f90ed       WALKWAY          005f90ed   

                                            geometry  
0  LINESTRING (-117.1706148251287 32.726548905684...  
1  LINESTRING (-117.1705245131968 32.726558517207...  
2  LINESTRING (-117.1666581391174 32.722214661862...  
3  LINESTRING (-117.166581746989 32.7222178855665...  
4  LINESTRING (-117.1665182431566 32.723810889902...

In [5]:
assets.head()

assetUid assetType  \
0  000223ee-a868-474b-abcb-12ff1bad00a3    CAMERA   
1  0002e3bb-5a9a-4083-9a96-fad0d22877b9       MIC   
2  0003806e-e409-449f-aab1-fc95f6e88d4e    CAMERA   
3  000a4ac7-224b-4fb7-a2e2-cd0592d2b2de    CAMERA   
4  000b2365-5309-422a-9be6-1b7127ca18db      NODE   

                         parentAssetUid    mediaType events  \
0  131d67b7-daaf-4252-9202-5df347a6c2a3  IMAGE,VIDEO          
1  e8beafbf-7836-48c3-bb14-440970eecd6d        AUDIO          
2  f91ec557-f03e-4189-8b83-5aa8d28fbf14  IMAGE,VIDEO          
3  741ce488-72fa-4216-a65d-216f7bdf8b7d  IMAGE,VIDEO          
4                                  None         None          

                                     geometry  
0            POINT (-117.1600173 32.71143062)  
1            POINT (-117.1493049 32.70997207)  
2            POINT (-117.1566029 32.70934863)  
3             POINT (-117.148666 32.71585673)  
4  POINT (-117.264282290807 32.8105285177251)

In [7]:
parking = [ e for e in loc_rec if e.locationType == 'PARKING_ZONE']

In [8]:
import metapack as mp
pkg = mp.open_package('http://library.metatab.org/sandiegodata.org-geography-2018.csv')
pkg

In [9]:
tracts = pkg.resource('tract_boundaries').geoframe()
tract_comm =  pkg.resource('tract_communities').dataframe()

In [11]:
t = gpd.GeoDataFrame(
    [ (e.locationUid, e.geometry) for e in parking],
    columns = ['locationUid','geometry'],
    geometry='geometry')
t.crs = tracts.crs
t['geometry'] = t.geometry.centroid

t = gpd.sjoin(t, tracts)
parking_geo = t.merge(tract_comm, on='geoid').drop(columns=['geometry', 'index_right'])
parking_geo.head()

locationUid               geoid  city_name city_code  \
0  03e23a6ace01fc14a86ab250ce65b682  14000US00000006974  San Diego        SD   
1              0ni87m8vfbfkjixihah5  14000US00000006974  San Diego        SD   
2  17389570219b9f86583906e1adbe1f4c  14000US00000006974  San Diego        SD   
3  389a8ee517722dab58216eab0f617856  14000US00000006974  San Diego        SD   
4               3d87wafg2r6jixhhy9t  14000US00000006974  San Diego        SD   

  community_name  community_cpcode community_code  
0       Downtown                 4            DOW  
1       Downtown                 4            DOW  
2       Downtown                 4            DOW  
3       Downtown                 4            DOW  
4       Downtown                 4            DOW

In [ ]:
parking_geo.to_csv('parking_geo.csv')

In [ ]:
com = pkg.resource('sd_community_boundaries').geoframe()
com.plot()

In [ ]:
com = com.to_crs({'init':'epsg:2875'})
com.plot()

In [ ]:
pkg = mp.jupyter.open_package()
roads = pkg.resource('all_roads').geoframe()

sdroads = roads[(roads.lpsjur == 'SD') | (roads.rpsjur == 'SD')].copy()

# Convert to EPSG:2875 ( California zone 6, feet ) so we can make the buffer in feet
sdroads = sdroads.to_crs({'init':'epsg:2875'})

# Buffer the roads to be big enough to enclose the assets
sdroads['geometry'] = sdroads.geometry.apply(lambda g : g.buffer(60,cap_style=1, join_style=2))
